# Origins to Closest Destinations Tutorial
This tutorial demonstrates how to run an **Origins to Closest Destinations (OD) analysis** with RA2CE.
RA2CE automatically finds the shortest or quickest route from each origin to its nearest destination.

If you are not yet familiar with preparing origins and destinations shapefiles, see the [Origins and Destinations Data Preparation](../tutorials/accessibility.prepare_data_origin_destinations.html) tutorial.

## Step 1: Import Libraries and Set Paths

In [1]:
from pathlib import Path

from ra2ce.analysis.analysis_config_data.analysis_config_data import AnalysisSectionLosses, AnalysisConfigData
from ra2ce.analysis.analysis_config_data.enums.analysis_losses_enum import AnalysisLossesEnum
from ra2ce.analysis.analysis_config_data.enums.weighing_enum import WeighingEnum
from ra2ce.network.network_config_data.enums.aggregate_wl_enum import AggregateWlEnum
from ra2ce.network import RoadTypeEnum
from ra2ce.network.network_config_data.enums.network_type_enum import NetworkTypeEnum
from ra2ce.network.network_config_data.enums.source_enum import SourceEnum
from ra2ce.network.network_config_data.network_config_data import (
    NetworkSection, NetworkConfigData, OriginsDestinationsSection, HazardSection
)
from ra2ce.ra2ce_handler import Ra2ceHandler

# Specify the path to your RA2CE project folder and input data
root_dir = Path('data', 'closest_origin_destinations')
network_path = root_dir.joinpath('static', 'network')

c:\Users\hauth\miniforge3\envs\ra2ce_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Step 2: Define Network with Origins & Destinations
Define the network configuration using OSM data clipped to a region polygon and include specific road types.

In [2]:
network_section = NetworkSection(
    source=SourceEnum.OSM_DOWNLOAD,
    polygon=network_path.joinpath("region_polygon.geojson"),
    network_type=NetworkTypeEnum.DRIVE,
    road_types=[
        RoadTypeEnum.MOTORWAY,
        RoadTypeEnum.MOTORWAY_LINK,
        RoadTypeEnum.PRIMARY,
        RoadTypeEnum.PRIMARY_LINK,
        RoadTypeEnum.SECONDARY,
        RoadTypeEnum.SECONDARY_LINK,
        RoadTypeEnum.TERTIARY,
        RoadTypeEnum.TERTIARY_LINK,
        RoadTypeEnum.RESIDENTIAL,
    ],
    save_gpkg=True,
)

In [3]:
origin_destination_section = OriginsDestinationsSection(
    origins=network_path.joinpath("origins.shp"),
    destinations=network_path.joinpath("destinations.shp"),
    origins_names="A",
    destinations_names="B",
    origin_count="POPULATION",
)

In [4]:
hazard_section = HazardSection(
    hazard_map=[root_dir.joinpath("static", "hazard", "max_flood_depth.tif")],
    aggregate_wl=AggregateWlEnum.MEAN,
    hazard_crs="EPSG:32736",
    overlay_segmented_network=False,
)

In [5]:
network_config_data = NetworkConfigData(
    root_path=root_dir,
    static_path=root_dir.joinpath('static'),
    network=network_section,
    hazard=hazard_section,
    origins_destinations=origin_destination_section,
)

## Step 3: Define the Analysis
Use [AnalysisLossesEnum.MULTI_LINK_ORIGIN_CLOSEST_DESTINATION](../api/ra2ce.analysis.analysis_config_data.enums.html#module-ra2ce.analysis.analysis_config_data.enums.analysis_losses_enum){.api-ref} to calculate routes that avoid disrupted roads.

In [6]:
analyse_section = AnalysisSectionLosses(
    name="OD_accessibility_analysis",
    analysis=AnalysisLossesEnum.MULTI_LINK_ORIGIN_CLOSEST_DESTINATION,
    weighing=WeighingEnum.LENGTH,
    calculate_route_without_disruption=True,
    save_csv=True,
    save_gpkg=True,
)

analysis_config_data = AnalysisConfigData(
    output_path=root_dir.joinpath("output"),
    static_path=root_dir.joinpath('static'),
    analyses=[analyse_section],
)

## Step 4: Run the Analysis

In [7]:
handler = Ra2ceHandler.from_config(
    network=network_config_data,
    analysis=analysis_config_data
)
handler.configure()
handler.run_analysis()

c:\Users\hauth\miniforge3\envs\ra2ce_env\Lib\site-packages\osmnx\simplification.py:513: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  merged = convert.graph_to_gdfs(G, edges=False)["geometry"].buffer(tolerance).unary_union
c:\Users\hauth\miniforge3\envs\ra2ce_env\Lib\site-packages\osmnx\simplification.py:560: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroids = node_clusters.centroid
100%|██████████| 4182/4182 [00:00<00:00, 376221.59it/s]
2025-10-02 10:44:34 AM - [avg_speed_calculator.py:175] - root - WARNING - No valid file found with average speeds data\closest_origin_destinations\static\output_graph\avg_speed.csv, calculating and saving them instead.
2025-10-02 10:44:34 AM - [avg_speed_calculator.py:150

[AnalysisResultWrapper(results_collection=[AnalysisResult(analysis_result=      o_id                    geometry   POPULATION  EV1_me   EV1_me_A
 0      A_0  POINT (34.83703 -19.81738)   164.245378     0.0     access
 1      A_1  POINT (34.84120 -19.81738)   629.302981     0.0  no access
 2      A_2  POINT (34.84536 -19.81738)   765.095154     0.0  no access
 3      A_3  POINT (34.84953 -19.81738)   962.088997     0.0  no access
 4      A_4  POINT (34.85370 -19.81738)  1978.129044     0.0  no access
 ..     ...                         ...          ...     ...        ...
 105  A_105  POINT (34.87036 -19.85071)   889.956332     0.0     access
 106  A_106  POINT (34.87453 -19.85071)   782.924690     0.0     access
 107  A_107  POINT (34.87870 -19.85071)  1259.180931     0.0     access
 108  A_108  POINT (34.88287 -19.85071)   953.192947     0.0     access
 109  A_109  POINT (34.88703 -19.85071)   428.394680     0.0     access
 
 [110 rows x 5 columns], analysis_config=AnalysisSectionLosse

## Step 5: Interpret Results
Results are stored in the `output` folder and include both CSV and GeoPackage files.

In [8]:
import geopandas as gpd

analysis_output_path = root_dir / "output" / "multi_link_origin_closest_destination"
results_gpkg = analysis_output_path / "OD_accessibility_analysis_optimal_routes_without_hazard.gpkg"
gdf = gpd.read_file(results_gpkg)
gdf.head()

,o_node,d_node,origin,destination,lengthNorm,origin_cnt,category,geometry
0,683300823,1934244652,A_56,B_2,796.051,1509.235690,special,"MULTILINESTRING ((34.84040 -19.83783, 34.84214..."
1,762948405,1934244652,A_72,B_2,608.413,1494.482965,special,"MULTILINESTRING ((34.84105 -19.84204, 34.84235..."
2,776490175,5632424987,A_43,B_0,1250.467,1202.655342,special,"MULTILINESTRING ((34.85082 -19.83083, 34.85079..."
3,776513552,5632424987,A_29,B_0,1304.066,1691.792524,special,"MULTILINESTRING ((34.84576 -19.82974, 34.84538..."
4,776513559,5632424987,A_30,B_0,902.755,2140.271343,special,"MULTILINESTRING ((34.84979 -19.83011, 34.84988..."


### Identifying Isolated Populations
Origins that cannot reach any destination due to hazard disruption are flagged in `OD_accessibility_analysis_origins.gpkg`.

In [9]:
origin_gdf = gpd.read_file(analysis_output_path / 'OD_accessibility_analysis_origins.gpkg')
map = origin_gdf.explore(column='EV1_me_A', cmap=['green', 'red'],
                         marker_kwds={'radius':5}, tiles="CartoDB dark_matter")
map.save("access_POP.html")

In [10]:
no_access_gdf = origin_gdf[origin_gdf['EV1_me_A'] == 'no access']
no_access_gdf.explore(column='POPULATION', cmap='cool',
                      marker_kwds={'radius':5}, tiles="CartoDB dark_matter")

### Inspecting Optimal Routes
Routes are computed from each origin to its closest destination. Routes can be filtered by specific destinations or categories.

In [16]:
destinations_gdf
b_6_gdf
optimal_routes_with_hazard_gdf


,o_node,d_node,origin,destination,lengthDisr,origin_cnt,category,name,lengthNorm,difference,geometry
0,683300823,1934244652,A_56,B_2,796.051,1509.235690,special,EV1_me,796.051,0.0,"MULTILINESTRING ((34.84040 -19.83783, 34.84214..."
1,762948405,1934244652,A_72,B_2,608.413,1494.482965,special,EV1_me,608.413,0.0,"MULTILINESTRING ((34.84105 -19.84204, 34.84235..."
2,776490175,1934244652,A_43,B_2,2644.836,1202.655342,special,EV1_me,NaN,NaN,"MULTILINESTRING ((34.84942 -19.83396, 34.84976..."
3,776513552,1934244652,A_29,B_2,3455.711,1691.792524,special,EV1_me,NaN,NaN,"MULTILINESTRING ((34.84576 -19.82974, 34.84538..."
4,776513559,1934244652,A_30,B_2,3209.712,2140.271343,special,EV1_me,NaN,NaN,"MULTILINESTRING ((34.85073 -19.83015, 34.85002..."
...,...,...,...,...,...,...,...,...,...,...,...
109,12233733643,12233733651,A_102,"B_5,B_6",2384.463,542.765158,special,EV1_me,2384.463,0.0,"MULTILINESTRING ((34.89489 -19.84609, 34.89456..."
110,12233733644,12233733651,A_106,"B_5,B_6",67.000,782.924690,special,EV1_me,67.000,0.0,"MULTILINESTRING ((34.87381 -19.85104, 34.87444..."
111,12233733645,12233733651,A_107,"B_5,B_6",545.948,1259.180931,special,EV1_me,545.948,0.0,"MULTILINESTRING ((34.87875 -19.85049, 34.87780..."
112,12233733646,12233733651,A_108,"B_5,B_6",989.799,953.192947,special,EV1_me,989.799,0.0,"MULTILINESTRING ((34.88230 -19.85140, 34.88280..."


In [ ]:
destinations_gdf = gpd.read_file(analysis_output_path / 'OD_accessibility_analysis_destinations.gpkg')
optimal_routes_with_hazard_gdf = gpd.read_file(analysis_output_path / 'OD_accessibility_analysis_optimal_routes_with_hazard.gpkg')

b_6_gdf = destinations_gdf[destinations_gdf['d_id'] == 'B_6']
optimal_routes_b_6_with_hazard_gdf = optimal_routes_with_hazard_gdf[optimal_routes_with_hazard_gdf['destination'] == 'B_6']
origins_with_optimal_route_b_6 = origin_gdf[origin_gdf['o_id'].isin(optimal_routes_b_6_with_hazard_gdf['origin'])]

optimal_routes_b_6_with_hazard_gdf.explore(column='difference',
                                           cmap='RdYlGn_r', legend=True,
                                           tiles="CartoDB dark_matter")

In [20]:
optimal_routes_with_hazard_gdf['destination'].unique()


array(['B_2', 'B_5,B_6', 'B_4', 'B_1'], dtype=object)

### Key Notes
- Some routes may no longer exist if disrupted roads block all access
- Remaining routes may be longer or slower, showing detours
- Some origins may completely lose access to the destination

This analysis helps quantify **loss of accessibility** due to hazards.